In [1]:
import isaac.constants
isaac.constants.TQDM_DISABLE = True

from torch import nn
from torch.nn import Softmax
from isaac.utils import get_cuda_device_if_available
import joblib

from isaac.dataset import read_dataset, prepare_dataset
from isaac.models import MultiBranchModel, ComplexRNNModel
from isaac.constants import BASIC_TRAINING_COLS, MASS_CLASS_COLS, FORCE_CLASS_COLS, RTHETA_COLS, XY_RTHETA_COLS, XY_VXVY_RTHETA_COLS
from isaac.evaluation import predict_with_a_group_of_saved_models, evaluate_saved_model

import torch
import glob
from torch.autograd import Variable
import numpy as np
import pandas as pd
from tqdm import tqdm


In [2]:
SECONDS_PER_WINDOW = 5 # seconds
FPS = 60
STEP_SIZE = 3
# PD_STEP_SIZE = 10
SEQ_END = 2700

In [3]:
device = get_cuda_device_if_available()
print(device)

cpu


In [4]:
normalise_data = True
model_name = "xy_vxvy_rtheta"
scaler_path = "scalers/passive_"+model_name+"_scaler.sk"
training_columns = XY_VXVY_RTHETA_COLS
network_dims = (len(training_columns), 25, 3, 0.5)
dataset_path = "../new_exp_data/exp7_passive_rtheta.h5"
class_columns = [list(MASS_CLASS_COLS), list(FORCE_CLASS_COLS)]
multiclass = True
DATASET = read_dataset(dataset_path)


def get_question_predictions_for_group_of_models(question_type):    
    models = sorted(glob.glob("models/"+model_name+"/best_"+question_type+"_model_seed_*.pt"))

    group_predictions = []
    
    predictions = predict_with_a_group_of_saved_models(tqdm(models), network_dims, None, 
                                                       training_columns=training_columns, 
                                                       class_columns=class_columns, step_size=STEP_SIZE, 
                                                       seq_end=SEQ_END, scaler_path=scaler_path,
                                                       arch=MultiBranchModel, multiclass=multiclass, trials=DATASET,
                                                       predict_rolling_windows=True, seconds_per_window=SECONDS_PER_WINDOW)

    predictions = torch.stack(predictions)
    
    print(predictions.shape)
    if question_type == "mass":
        predictions = predictions[:, :, :, 0]
    else:
        predictions = predictions[:, :, :, 1]

    return predictions

def get_question_accuracy_for_group_of_models(question_type):    
    model_paths = tqdm(sorted(glob.glob("models/"+model_name+"/best_"+question_type+"_model_seed_*.pt")))

    accuracies, predicted = evaluate_saved_model(model_paths, network_dims, dataset_path, 
                                                 training_columns=training_columns, class_columns=class_columns, 
                                                 step_size=STEP_SIZE, seq_end=SEQ_END, scaler_path=scaler_path,
                                                 arch=MultiBranchModel, multiclass=multiclass, trials=None)
    
    if question_type == "mass":
        question_index = 0
    else:
        question_index = 1

    accuracies = np.stack(accuracies)[:, question_index]
    
    predicted = [x[:, question_index].numpy() for x in predicted]

    return accuracies, predicted

# T-test for MASS questions

## Load model's predictions

In [5]:
print("MASS")
question_type = "mass"
group_mass_seq_prediction = get_question_predictions_for_group_of_models(question_type)

print("\nFORCE")
question_type = "force"
group_force_seq_prediction = get_question_predictions_for_group_of_models(question_type)

  0%|          | 0/25 [00:00<?, ?it/s]

MASS


/home/hector/miniconda3/envs/diss/lib/python3.5/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
  0%|          | 0/25 [00:00<?, ?it/s]/home/hector/miniconda3/envs/diss/lib/python3.5/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


torch.Size([25, 36, 40, 2, 3])

FORCE


100%|██████████| 25/25 [00:58<00:00,  2.20s/it]

torch.Size([25, 36, 40, 2, 3])


In [6]:
mass_solutions = [trial[list(MASS_CLASS_COLS)].idxmax(axis=1).unique()[0] for trial in DATASET]
force_solutions = [trial[list(FORCE_CLASS_COLS)].idxmax(axis=1).unique()[0] for trial in DATASET]

In [7]:
s = Softmax(dim=-1)
group_force_seq_prediction = s(group_force_seq_prediction)
group_mass_seq_prediction = s(group_mass_seq_prediction)

In [8]:
avg_force_seq_prediction = torch.mean(group_force_seq_prediction, dim=0)
avg_mass_seq_prediction = torch.mean(group_mass_seq_prediction, dim=0)

In [9]:
avg_force_seq_prediction.shape

torch.Size([36, 40, 3])

In [10]:
n_trials = avg_force_seq_prediction.shape[0]
n_windows = avg_force_seq_prediction.shape[1]
window_second_start = [i for _ in range(n_trials) for i in range(1, n_windows+1)]
trial_number = [i for i in range(n_trials) for _ in range(1, n_windows+1)]

In [11]:
mass_df = pd.DataFrame(data=avg_mass_seq_prediction.reshape(n_trials*n_windows, 3).numpy(), 
                       columns=["rnn_%s" % cl for cl in MASS_CLASS_COLS])
mass_df["window_second_start"] = window_second_start
mass_df["trial_number"] = trial_number
mass_df["solution"] = [mass_solutions[trial_id] for trial_id in trial_number]

force_df = pd.DataFrame(data=avg_force_seq_prediction.reshape(n_trials*n_windows, 3).numpy(), 
                       columns=["rnn_%s" % cl for cl in FORCE_CLASS_COLS])
force_df["window_second_start"] = window_second_start
force_df["trial_number"] = trial_number
force_df["solution"] = [force_solutions[trial_id] for trial_id in trial_number]

In [12]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(font_scale=3)
sns.set_style("white")

# Obtain most obvious intervals
Most obvious = Highest values of probability for a correct class

In [13]:
def make_probability_of_the_incorrect_rnn_column_zero(df):
    
    for solution in df.solution.unique():
        df["rnn_" + solution] = df["rnn_" + solution] * (df["solution"] == solution).astype(int)

def get_obvious_probabilities_df(df):
    obvious_dfs = []

    for solution in df.solution.unique():
        reshaped_df = df[["window_second_start", "trial_number", "solution"]].copy()
        reshaped_df["rnn_option"] = solution
        reshaped_df["rnn_value"] = df["rnn_" + solution]
        obvious_dfs.append(reshaped_df)

    obvious_dfs = pd.concat(obvious_dfs)
    obvious_dfs = obvious_dfs.query("solution == rnn_option")
    return obvious_dfs
    
make_probability_of_the_incorrect_rnn_column_zero(mass_df)
obvious_mass_dfs = get_obvious_probabilities_df(mass_df)
make_probability_of_the_incorrect_rnn_column_zero(force_df)
obvious_force_dfs = get_obvious_probabilities_df(force_df)

In [14]:
obvious_mass_dfs = obvious_mass_dfs.sort_values(by="rnn_value", ascending=False)
obvious_force_dfs = obvious_force_dfs.sort_values(by="rnn_value", ascending=False)

## Load human results

In [15]:
from isaac.visualization import make_frame_curried
import moviepy.editor as mpy
from scipy import misc


def make_clip(trial_data, window_second_start, solution, rnn_thinks_this_is, rnn_confidence):
            
    trial_data = trial_data.iloc[window_second_start*FPS:(window_second_start + SECONDS_PER_WINDOW)*FPS]
    duration = len(trial_data)
    n_bodies = sum(["o"+str(i)+".x" in list(trial_data.columns) for i in range(1, 5)])
    
    while (len(trial_data) + 1) % 60 != 0:
        trial_data = trial_data.append(trial_data.iloc[-1], ignore_index=True)
        
    make_frame = make_frame_curried(trial_data, n_bodies, None, None)
    clip = mpy.VideoClip(make_frame, duration=duration / 60)
    return clip, trial_data

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [16]:
%load_ext autoreload
%autoreload 2

# Create videos for comparing evidence seen by RNN and by IO

In [17]:
import os
import json

OBVIOUS_DATA_PATH = "cogsci_images/obvious_videos/obvious_%s" % model_name
os.makedirs(OBVIOUS_DATA_PATH, exist_ok=True)
FILENAME = "obvious_%s_interval_in_trial_%d_sec_%d_to_%d_rnn_thinks_%s_while_solution_is_%s"
VIDEO_PATH = os.path.join(OBVIOUS_DATA_PATH, FILENAME + ".mp4")
JSON_PATH = os.path.join(OBVIOUS_DATA_PATH, "obvious_"+model_name+"_%s_physics_data.json")

def write_obvious_intervals(confused_df, question_type):

    replays = read_dataset("../new_exp_data/exp7_passive.h5")
    print(len(replays))
    written_replays = {}
    number_of_written_replays = 0
    
    json_data = []

    for row_i in range(confused_df.shape[0]):

        window_second_start, trial_number, solution, rnn_thinks_this_is, rnn_confidence = (
            confused_df.iloc[row_i][["window_second_start", "trial_number", "solution", "rnn_option", "rnn_value"]])

        print("RNN thinks the interval (%d, %d) in trial %d is %s with %.4f confidence. In reality, it is %s." % (
            window_second_start, window_second_start + SECONDS_PER_WINDOW, trial_number, rnn_thinks_this_is, rnn_confidence, solution))

        if trial_number in written_replays:
            overlapping_replay = False
            for already_written_window_start in written_replays[trial_number]:
                if abs(already_written_window_start - window_second_start) <= 5:
                    overlapping_replay = True
                    break
            if overlapping_replay:
                print("Skipping trial %d and start %d" % (trial_number, window_second_start))
                continue

        written_replays[trial_number] = written_replays.get(trial_number, []) + [window_second_start]
        clip, trial_data = make_clip(replays[trial_number], window_second_start, solution, rnn_thinks_this_is, rnn_confidence)
        clip.ipython_display(fps=60)
        clip.write_videofile(VIDEO_PATH % (
            question_type, trial_number, window_second_start, window_second_start+SECONDS_PER_WINDOW,
            rnn_thinks_this_is, solution), fps=60)

        trial_data = trial_data.to_dict(orient='list')
        # Simplify attributes whose values are unique throughout the list
        for key in ["trial_type", "condition_world_variant", "tM", "tR", "world_id"]:
            trial_data[key] = trial_data[key][0]
        json_data.append(trial_data)

        number_of_written_replays += 1
        if number_of_written_replays == 10:
            break
            
    with open(JSON_PATH % question_type, "w+") as f: 
        json.dump(json_data, f)

In [18]:
write_obvious_intervals(obvious_force_dfs, "force")

36
RNN thinks the interval (27, 32) in trial 18 is attract with 0.9949 confidence. In reality, it is attract.


100%|█████████▉| 300/301 [00:02<00:00, 141.69it/s]


[MoviePy] >>>> Building video cogsci_images/obvious_videos/obvious_xy_vxvy_rtheta/obvious_force_interval_in_trial_18_sec_27_to_32_rnn_thinks_attract_while_solution_is_attract.mp4
[MoviePy] Writing video cogsci_images/obvious_videos/obvious_xy_vxvy_rtheta/obvious_force_interval_in_trial_18_sec_27_to_32_rnn_thinks_attract_while_solution_is_attract.mp4


100%|█████████▉| 300/301 [00:02<00:00, 138.71it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: cogsci_images/obvious_videos/obvious_xy_vxvy_rtheta/obvious_force_interval_in_trial_18_sec_27_to_32_rnn_thinks_attract_while_solution_is_attract.mp4 



...

RNN thinks the interval (21, 26) in trial 2 is attract with 0.9676 confidence. In reality, it is attract.
Skipping trial 2 and start 21
RNN thinks the interval (27, 32) in trial 11 is attract with 0.9675 confidence. In reality, it is attract.
Skipping trial 11 and start 27
RNN thinks the interval (25, 30) in trial 0 is attract with 0.9668 confidence. In reality, it is attract.
Skipping trial 0 and start 25
RNN thinks the interval (26, 31) in trial 11 is attract with 0.9654 confidence. In reality, it is attract.
Skipping trial 11 and start 26
RNN thinks the interval (17, 22) in trial 11 is attract with 0.9605 confidence. In reality, it is attract.
Skipping trial 11 and start 17
RNN thinks the interval (32, 37) in trial 18 is attract with 0.9580 confidence. In reality, it is attract.
Skipping trial 18 and start 32
RNN thinks the interval (21, 26) in trial 1 is attract with 0.9576 confidence. In reality, it is attract.
Skipping trial 1 and start 21
RNN thinks the interval (16, 21) in tria

100%|█████████▉| 300/301 [00:02<00:00, 142.31it/s]

[MoviePy] >>>> Building video cogsci_images/obvious_videos/obvious_xy_vxvy_rtheta/obvious_force_interval_in_trial_28_sec_16_to_21_rnn_thinks_attract_while_solution_is_attract.mp4
[MoviePy] Writing video cogsci_images/obvious_videos/obvious_xy_vxvy_rtheta/obvious_force_interval_in_trial_28_sec_16_to_21_rnn_thinks_attract_while_solution_is_attract.mp4



100%|█████████▉| 300/301 [00:02<00:00, 141.42it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: cogsci_images/obvious_videos/obvious_xy_vxvy_rtheta/obvious_force_interval_in_trial_28_sec_16_to_21_rnn_thinks_attract_while_solution_is_attract.mp4 



In [19]:
write_obvious_intervals(obvious_mass_dfs, "mass")

36
RNN thinks the interval (17, 22) in trial 28 is A with 0.9553 confidence. In reality, it is A.


100%|█████████▉| 300/301 [00:02<00:00, 145.29it/s]


[MoviePy] >>>> Building video cogsci_images/obvious_videos/obvious_xy_vxvy_rtheta/obvious_mass_interval_in_trial_28_sec_17_to_22_rnn_thinks_A_while_solution_is_A.mp4
[MoviePy] Writing video cogsci_images/obvious_videos/obvious_xy_vxvy_rtheta/obvious_mass_interval_in_trial_28_sec_17_to_22_rnn_thinks_A_while_solution_is_A.mp4


100%|█████████▉| 300/301 [00:02<00:00, 140.08it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: cogsci_images/obvious_videos/obvious_xy_vxvy_rtheta/obvious_mass_interval_in_trial_28_sec_17_to_22_rnn_thinks_A_while_solution_is_A.mp4 

RNN thinks the interval (16, 21) in trial 11 is B with 0.9509 confidence. In reality, it is B.


...

RNN thinks the interval (28, 33) in trial 11 is B with 0.9023 confidence. In reality, it is B.
Skipping trial 11 and start 28
RNN thinks the interval (37, 42) in trial 19 is A with 0.9007 confidence. In reality, it is A.
Skipping trial 19 and start 37
RNN thinks the interval (3, 8) in trial 5 is B with 0.9005 confidence. In reality, it is B.
Skipping trial 5 and start 3
RNN thinks the interval (2, 7) in trial 5 is B with 0.8996 confidence. In reality, it is B.
Skipping trial 5 and start 2
RNN thinks the interval (2, 7) in trial 32 is B with 0.8948 confidence. In reality, it is B.


100%|█████████▉| 300/301 [00:02<00:00, 142.50it/s]

[MoviePy] >>>> Building video cogsci_images/obvious_videos/obvious_xy_vxvy_rtheta/obvious_mass_interval_in_trial_32_sec_2_to_7_rnn_thinks_B_while_solution_is_B.mp4
[MoviePy] Writing video cogsci_images/obvious_videos/obvious_xy_vxvy_rtheta/obvious_mass_interval_in_trial_32_sec_2_to_7_rnn_thinks_B_while_solution_is_B.mp4



100%|█████████▉| 300/301 [00:02<00:00, 139.02it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: cogsci_images/obvious_videos/obvious_xy_vxvy_rtheta/obvious_mass_interval_in_trial_32_sec_2_to_7_rnn_thinks_B_while_solution_is_B.mp4 

